In [ ]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
import timm
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
import PIL


import matplotlib.pyplot as plt

In [ ]:
test_dir = '/home/hj/Study/MLops/Test_Data'

In [ ]:
class CNN(nn.Module):
    """
    Base Implement Model to training
    """
    def __init__(self, class_n, rate=0.1):
        super(CNN, self).__init__()
        self.model = models.resnet50(pretrained=True)
        self.classifier1 = nn.Linear(1000,25)
        self.dropout = nn.Dropout(rate)
    
    def forward(self, inputs):
        output = self.model(inputs)
        output = self.dropout(self.classifier1(output))# dropout 조심.
        return output

In [ ]:
A_transform = {
    "train_Filp_transform": A.Compose(
        [
            A.OneOf(
                [
                    A.HorizontalFlip(p=1.0),
                    A.VerticalFlip(p=1.0),
                    A.RandomRotate90(p=1.0),
                ],
                p=0.75,
            ),
            A.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=0.5, p=1),
            ToTensorV2(p=1.0),  # dtype float32, transpose 한번에
        ]
    ),
    "train_deault_transform": A.Compose([ToTensorV2(p=1.0)]),
}


In [ ]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = PIL.Image.open(self.img_paths[index])
        image = np.array(image.convert("RGB"))
        if self.transform:
            image = self.transform(image=image)
            image = image['image']
        return image

    def __len__(self):
        return len(self.img_paths)

In [ ]:
from tqdm import tqdm
from glob import glob

image_paths = sorted(glob(os.path.join(test_dir, "*/*.jpg")))
dataset = TestDataset(
    image_paths, transform=A_transform["train_Filp_transform"]
)


In [ ]:
print(len(image_paths))

In [ ]:
fig, axes = plt.subplots(1,1,figsize=(5,5))
print(dataset[3].shape)
plt.imshow(dataset[3].permute(1,2,0))
plt.show()
